# Corpo isolado uniformemente magnetizado

Este código está escrito em linguagem [Python](https://www.python.org/) e utiliza o subpacote [gravmag](http://www.fatiando.org/api/gravmag.html#module-fatiando.gravmag) do [Fatiando a Terra](http://www.fatiando.org/index.html).

### Pacotes necessários

In [1]:
from fatiando import gridder, mesher, utils
from fatiando.gravmag import prism, imaging
from fatiando.vis import mpl, myv

/home/vanderlei/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/vanderlei/anaconda2/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Dados sintéticos produzido por um modelo 3D

In [2]:
# Make some synthetic gravity data from a simple prism model
model = [mesher.Prism(-4000, 0, -4000, -2000, 2000, 5000, {'density': 1200}),
         mesher.Prism(-1000, 1000, -1000, 1000, 1000, 7000, {'density': -800}),
         mesher.Prism(2000, 4000, 3000, 4000, 0, 2000, {'density': 600})]
# Calculate on a scatter of points to show that migration doesn't need gridded
# data
xp, yp, zp = gridder.scatter((-6000, 6000, -6000, 6000), 1000, z=-10)
gz = utils.contaminate(prism.gz(xp, yp, zp, model), 0.1)

In [3]:
#mesh = imaging.migrate(xp, yp, zp, gz, 0, 10000, (30, 30, 30), power=0.8)

In [4]:
# Plot the results
myv.figure()
myv.prisms(model, 'density', style='wireframe', linewidth=2)
#myv.prisms(mesh, 'density', edges=False)
axes = myv.axes(myv.outline())
myv.wall_bottom(axes.axes.bounds)
myv.wall_north(axes.axes.bounds)
myv.show()

ImportError: No module named enthought.mayavi